# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Общая информация данных

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [8]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [9]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [10]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

**Вывод**

1. - children - максимальные и минимальные значения очень интересные;
2 - days_employed - общий трудовой стаж отрицательный, либо очень большой;
3 - days_employed - есть пропущенные значения, проверить связь со столбцом total_income, их количество одинаковое
4. - dob_years - минимальный возраст клиента 0;
5. - education - все значения написаны в разном регистре, кто-то забыл выключить Caps;
6. - education_id, family_status, family_status_id, income_type - значения в столбцах выглядят нормальными;

## Предобработка данных

In [11]:
#У нас 47 значений (-1) и 76 значений 20.
#В первом случае возможно это означает пропуск и стоит заменить на ноль.
#Во втором случае, возможно случился какой-то баг и значения умножились, значения стоит заменить на два.
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [12]:
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 2)

In [13]:
#нельзя определить возраст человека, стоит удалить строки с таким возрастом это 0,45% от всего датафрейма
df['dob_years'][df['dob_years'] < 20].value_counts()

0     101
19     14
Name: dob_years, dtype: int64

In [14]:
#фильтруем столбец dob_years
df = df.loc[df['dob_years'] >0]

In [15]:
#преоброзуем поля с образованием в нижний регистр
df['education'] = df['education'].str.lower()

In [16]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [17]:
#странное значение пола, удалим его
df['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

In [18]:
#фильтруем столбец gender 
df = df.loc[df['gender'] != 'XNA']

In [19]:
#переведем столбец с образованием в нижний регистр
df['education'] = df['education'].str.lower()

In [20]:
#все ок
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Обработка пропусков

In [21]:
#все пропуски в days_employed связаны с пропусками в total_income
(df['days_employed'].isnull() == df['total_income'].isnull()).unique()


array([ True])

Разберем пропущенные значения в годовом доходе. Возможно пропуски возникли случайным образом. Стоит их заполнить значениями среднего по образованию и должности.

In [22]:
#заполняем пропуски
df['total_income'] = df.groupby(['income_type', 'education'])['total_income'].transform(lambda x: x.fillna(x.mean()))

In [23]:
#проверим количество положительных значений стажа
df[df['days_employed'] >0]['days_employed'].count()

3428

In [24]:
#посмотрим какой тип занятости у этих людей
df[df['days_employed'] >0]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [25]:
#и посмотрим тип занятости у людей с отрицательным заработком
df[df['days_employed'] <0]['income_type'].unique()

array(['сотрудник', 'компаньон', 'госслужащий', 'студент',
       'предприниматель', 'в декрете'], dtype=object)

In [26]:
df[df['days_employed'] >0]['days_employed'].isna().sum()

0

Заметим, что только у пенсионеров и безработных нет отрицательных значений, а у положительных значений нет пропусков.

In [27]:
#кажется среднее очень большое
df[df['days_employed'] >0]['days_employed'].mean()

365016.54314603505

In [28]:
#неплохо, с поднятием пенсионного возраста, люди стали работать по тысячу лет
df[df['days_employed'] >0]['days_employed'].mean()/365

1000.0453236877672

In [29]:
#проверим средний возраст этих людей, может ошибка в том, что подсчет стажа шел в часах
df[df['days_employed'] >0]['dob_years'].mean()

59.41802800466744

In [30]:
#средний возраст выглядит нормальным, тогда изменим значения стажа, поделив его на количество часов в сутках
df.loc[(df['days_employed'] > 0), 'days_employed'] = df.loc[(df['days_employed'] > 0), 'days_employed']/24

In [31]:
#теперь разберемся с отрицательными значениями, все значения преобразуем по модулю
df.loc[(df['days_employed'] < 0), 'days_employed'] = df.loc[(df['days_employed'] < 0), 'days_employed'].abs()

In [32]:
#отфильтруем стаж работы по среднему значению сгруппированному по типу занятости
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()))

In [33]:
df['days_employed'].isna().sum()

0

**Вывод**

Возможно некорректно заполнять пропущенные значения. Стоило бы узнать истинную причину пропусков, позвонив нескольким клиентам. Они могли просто не указать в заявке стаж, или недавно устроились на работу после долгого перерыва, поэтому в графе ежемесячного дохода стоит пропуск. Так как количество пропусков составляет около 10% от всех данных, что достаточно много, то было принято решений заменить пропуски на средние значения.

### Замена типа данных

In [34]:
#заменим тип данных в столбцах со стажем и зарплатой
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

**Вывод**

Изменен тип данных у столбцов со стажем и ежемесячным доходом.

### Обработка дубликатов

In [35]:
#посчитаем количество дупликатов в датафрейме
df.duplicated().sum()

71

In [36]:
#количество дубликатов небольшое, можно их удалить
df = df.drop_duplicates()

**Вывод**

Были удалены дубликаты, возможно было задвоение данных. Это сделано для более наглядного представления столбцов.

### Лемматизация

In [37]:
m = Mystem()
purpose_list = df['purpose'].unique()
purpose_list

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [38]:
lemmas_purpose = []
for row in purpose_list:
    lemmas_purpose += m.lemmatize(row)

Counter(lemmas_purpose)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

**Вывод**

Можно выделить основные цели кредитов - это:
1. Недвижимость
2. Образование
3. Свадьба
4. Автомобиль

### Категоризация данных

In [39]:
def dob_years_func(row):
    if row < 30:
        return 'до 30 лет'
    elif 30 <= row < 45:
        return '30-45 лет'
    elif 45 <= row < 65:
        return '45-65 лет'
    else:
        return 'старше 65 лет'

In [40]:
def children_func(children):
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return '1 ребенок'
    elif children == 2:
        return '2 ребенка'
    else:
        return '3 и более'

In [41]:
def purpose_func(str):
    lemmas = m.lemmatize(str)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'автомобиль' in lemmas:
        return 'автомобиль'   

In [42]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21352.000000,21352.000000,21352.000000,21352.000000,21352.000000,21352.000000,2.135200e+04
mean,0.478316,4647.735622,43.476817,0.817722,0.972649,0.081163,1.675318e+05
std,0.755786,5320.102803,12.241877,0.548717,1.421020,0.273092,9.839369e+04
min,0.000000,24.000000,19.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1023.000000,33.000000,1.000000,0.000000,0.000000,1.076482e+05
50%,0.000000,2328.000000,43.000000,1.000000,0.000000,0.000000,1.523040e+05
75%,1.000000,5320.250000,53.000000,1.000000,1.000000,0.000000,1.982935e+05
max,5.000000,18388.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [43]:
#Исходя из таблицы выше можно категоризировать уровень дохода
def income_status_func(monthly_income):
    if monthly_income <= 107000:
            return 'низкий'
    if monthly_income <= 152000:
            return 'средний'
    if monthly_income < 198000:
            return 'выше среднего'
    return 'высокий'

In [44]:
df['number_of_child'] = df['children'].apply(children_func)
df['purpose_category'] = df['purpose'].apply(purpose_func)
df['age_status'] = df['dob_years'].apply(dob_years_func)
df['income_status'] = df['total_income'].apply(income_status_func)

In [45]:
#посмотрим как выглядят новые столбцы
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,number_of_child,purpose_category,age_status,income_status
5632,0,2328,36,среднее,1,женат / замужем,0,F,сотрудник,0,152676,покупка коммерческой недвижимости,нет детей,недвижимость,30-45 лет,выше среднего
21232,0,1437,25,высшее,0,Не женат / не замужем,4,M,компаньон,0,136647,на покупку автомобиля,нет детей,автомобиль,до 30 лет,средний
6658,0,4425,31,среднее,1,женат / замужем,0,F,компаньон,0,189384,ремонт жилью,нет детей,недвижимость,30-45 лет,выше среднего
2743,0,2328,46,среднее,1,женат / замужем,0,F,сотрудник,0,152676,покупка жилья для сдачи,нет детей,недвижимость,45-65 лет,выше среднего
8616,0,15208,59,среднее,1,вдовец / вдова,2,F,пенсионер,0,131683,свой автомобиль,нет детей,автомобиль,45-65 лет,средний


**Вывод**

Теперь в таблице есть новые столбцы разделенные по категориям.

In [46]:
#преобразование в словарь
df_dict = {"family" : df['family_status'].unique(),
          "education" : df['education'].unique(),
          "number_of_child" : df['number_of_child'].unique(),
          "purpose_category" : df['purpose_category'].unique(),
          "income_status" : df['income_status'].unique()}
df_dict2= pd.DataFrame.from_dict(df_dict, orient = 'index')
df_dict2

,0,1,2,3,4
family,женат / замужем,гражданский брак,вдовец / вдова,в разводе,Не женат / не замужем
education,высшее,среднее,неоконченное высшее,начальное,ученая степень
number_of_child,1 ребенок,нет детей,3 и более,2 ребенка,None
purpose_category,недвижимость,автомобиль,образование,свадьба,None
income_status,высокий,средний,выше среднего,низкий,None


Выше приведено преобразование в словарь категоризированных столбцов.

## Зависимость нескольких параметров на возврат кредита в срок

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
children_debt = df.pivot_table(index='number_of_child',values='debt',aggfunc=['count','sum'])
ratio_children = children_debt[('sum', 'debt')]/children_debt[('count', 'debt')]
children_debt['ratio_children']=round(ratio_children*100,1)
children_debt

,count,sum,ratio_children
,debt,debt,
number_of_child,,,
1 ребенок,4792,441,9.2
2 ребенка,2114,202,9.6
3 и более,378,31,8.2
нет детей,14068,1059,7.5


**Вывод**

Лучшие заемщики без детей и таких большинство. По человеческой логике многодетные семьи должны быть нехорошими плательщиками кредитов, но видимо из-за большой ответственности и государственных льгот на многодетные семьи, ситуация у них гораздо лучше, нежели чем те у кого 1 или 2 ребенка. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
family_status_debt = df.pivot_table(index='family_status',values='debt',aggfunc=['count','sum'])
ratio_family_status = family_status_debt[('sum', 'debt')]/family_status_debt[('count', 'debt')]
family_status_debt['ratio_family_status']=round(ratio_family_status*100,1)
family_status_debt

,count,sum,ratio_family_status
,debt,debt,
family_status,,,
Не женат / не замужем,2794,273,9.8
в разводе,1185,85,7.2
вдовец / вдова,954,62,6.5
гражданский брак,4129,386,9.3
женат / замужем,12290,927,7.5


**Вывод**

Люди в официальном браке и после брака более ответственны к погашению кредита, а тех кто находится в гражданском браке или не женаты/не замужем. Это говорит об ответственности людей перед своей семьей, нежели перед самим собой.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
income_status_debt = df.pivot_table(index='income_status',values='debt',aggfunc=['count','sum'])
ratio_income = income_status_debt[('sum', 'debt')]/income_status_debt[('count', 'debt')]
income_status_debt['ratio_income']=round(ratio_income*100,1)
income_status_debt

,count,sum,ratio_income
,debt,debt,
income_status,,,
высокий,5359,374,7.0
выше среднего,5338,464,8.7
низкий,5279,422,8.0
средний,5376,473,8.8


**Вывод**

Очевидно видеть, что люди с высоким доходом лучше финансово образованы. Гораздо неочевиднее видеть людей с низкими доходами на втором месте, учитывая что часто такие люди с низкой социальной ответственностью и любят брать микрозаймы, кредитные карты и не отдавать деньги. Возможно у этого банка пугающие коллекторы, а возможно не хотят портить кредитную историю. Еще есть предположение, что это небольшие кредиты. Клиенты со средним доходом и доходом выше среднего меньше всего рассчитывают свои силы.  

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
purpose_category_debt = df.pivot_table(index='purpose_category',values='debt',aggfunc=['count','sum'])
ratio_purpose = purpose_category_debt[('sum', 'debt')]/purpose_category_debt[('count', 'debt')]
purpose_category_debt['ratio_purpose']=round(ratio_purpose*100,1)
purpose_category_debt

,count,sum,ratio_purpose
,debt,debt,
purpose_category,,,
автомобиль,4284,400,9.3
недвижимость,10763,779,7.2
образование,3995,370,9.3
свадьба,2310,184,8.0


**Вывод**

Люди более ответственно и осознано берут кредиты на недвижимость, чего не скажешь про автомобили и образование. Автомобили ломаются, приходится вкладывать деньги, а образование можно бросить - заставить ленивый мозг работать стоит усилий, а так не хочется.  

- В дополнение, хочется посмотреть как образование влияет на возврат кредита в срок

In [51]:
education_debt = df.pivot_table(index='education',values='debt',aggfunc=['count','sum'])
ratio_education = education_debt[('sum', 'debt')]/education_debt[('count', 'debt')]
education_debt['ratio_education']=round(ratio_education*100,1)
education_debt

,count,sum,ratio_education
,debt,debt,
education,,,
высшее,5215,277,5.3
начальное,282,31,11.0
неоконченное высшее,741,68,9.2
среднее,15108,1357,9.0
ученая степень,6,0,0.0


Тут все очевидно.

## Общий вывод

Лучшими заемищками можно считать клиентов без детей, с высшим образованием, женатых/замужних(либо переживших брак по причине смерти или развода),с высоким уровнем дохода и с целью совершения покупки или манипуляциями с недвижимостью.
Худшими заемщиками являются клиенты с 1-2 детьми, с начальным образованием, не находящиеся в официальных отношениях, со средним или выше среднего уровнем дохода и с целью покупки автомобиля или вложения средств в образование.